In [37]:
# Import the libraries
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import seaborn as sns  # for nicer plots
sns.set(style="darkgrid")  # default style

import tensorflow as tf
from tensorflow import keras
from keras import metrics
tf.get_logger().setLevel('INFO')

from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score, classification_report

In [38]:
#load data
X_train = pd.read_csv('../data/processed/X_train.csv')
Y_train = pd.read_csv('../data/processed/Y_train.csv')
X_val = pd.read_csv('../data/processed/X_val.csv')
Y_val = pd.read_csv('../data/processed/Y_val.csv')
X_test = pd.read_csv('../data/processed/X_test.csv')
Y_test = pd.read_csv('../data/processed/Y_test.csv')

### Implementing Baseline

In [39]:
#show class distribution
print(Y_train.value_counts())

music_category
0                 44460
3                  7242
2                  7190
4                  3625
1                  2958
5                  1755
6                  1171
Name: count, dtype: int64


Since class distribution isn't too unbalanced, implement majority baseline and stratified baseline.

In [40]:
#majority baseline implementation
dummy_clf = DummyClassifier(strategy = 'stratified', random_state = 42)
dummy_clf.fit(X_train, Y_train)
Y_pred = dummy_clf.predict(X_test)
accuracy = accuracy_score(Y_test, Y_pred)
report = classification_report(Y_test, Y_pred)

print(f'Baseline Classifier Accuracy: {accuracy}')
print(f'Classification Report: {report}')

Baseline Classifier Accuracy: 0.45344094039212246
Classification Report:               precision    recall  f1-score   support

           0       0.65      0.65      0.65     14744
           1       0.05      0.05      0.05      1033
           2       0.12      0.12      0.12      2388
           3       0.11      0.11      0.11      2407
           4       0.05      0.05      0.05      1154
           5       0.03      0.02      0.03       641
           6       0.02      0.02      0.02       432

    accuracy                           0.45     22799
   macro avg       0.15      0.15      0.15     22799
weighted avg       0.45      0.45      0.45     22799



### Building a Model

In [42]:
len(Y_train['music_category'].unique())

7

In [43]:
X_train.shape

(68401, 22)

In [44]:
def build_model(num_features, n_classes,learning_rate=0.01):
    """Build a TF logistic regression model using Keras.
    
    Args:
    learning_rate: The desired learning rate for SGD.
    
    Returns:
    model: A tf.keras model (graph).
    """
    # Random Seed + Clear Session
    tf.keras.backend.clear_session()
    np.random.seed(0)
    tf.random.set_seed(0)
    
    # Build a model using keras.Sequential.
    model = keras.Sequential(name = 'Genres')
    
    # Keras layers processing
    model.add(tf.keras.layers.InputLayer(input_shape=num_features))
    
    # This layer constructs the linear set of parameters for each input feature
    # (as well as a bias), and applies a sigmoid to the result. The result is
    # binary logistic regression.
    model.add(keras.layers.Dense(
      units=n_classes,
      activation='softmax'))

    # Use the SGD optimizer as usual.
    optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate)
    
    # We specify the binary_crossentropy loss (equivalent to log loss).
    # Notice that we are including 'binary accuracy' as one of the metrics that we
    # ask Tensorflow to report when evaluating the model.
    model.compile(loss='sparse_categorical_crossentropy', 
                optimizer=optimizer, 
                metrics=['accuracy'])
    
    return model

In [46]:
model = build_model(num_features = X_train.shape[1], n_classes = len(Y_train['music_category'].unique()))
model.summary()

Model: "Genres"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 7)                 161       
                                                                 
Total params: 161 (644.00 Byte)
Trainable params: 161 (644.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [47]:
# Fit the model.
history = model.fit(
  x = X_train,   # training examples
  y = Y_train,   #labels
  epochs=5,             # number of passes through the training data
  batch_size=64,        # mini-batch size for SGD
  validation_split=0.1, # use a fraction of the examples for validation -- DO WE STILL NEED THIS IF VALIDATION SPLIT
  verbose=1             # display some progress output during training
  )

# Convert the return value into a DataFrame so we can see the train loss 
# and binary accuracy after every epoch.
history = pd.DataFrame(history.history)
display(history)

Epoch 1/5
962/962 [==============================] - 1s 874us/step - loss: nan - accuracy: 0.5770 - val_loss: nan - val_accuracy: 0.6530
Epoch 2/5
962/962 [==============================] - 1s 776us/step - loss: nan - accuracy: 0.6497 - val_loss: nan - val_accuracy: 0.6530
Epoch 3/5
962/962 [==============================] - 1s 775us/step - loss: nan - accuracy: 0.6497 - val_loss: nan - val_accuracy: 0.6530
Epoch 4/5
962/962 [==============================] - 1s 766us/step - loss: nan - accuracy: 0.6497 - val_loss: nan - val_accuracy: 0.6530
Epoch 5/5
962/962 [==============================] - 1s 772us/step - loss: nan - accuracy: 0.6497 - val_loss: nan - val_accuracy: 0.6530


,loss,accuracy,val_loss,val_accuracy
0,NaN,0.577047,NaN,0.652975
1,NaN,0.649659,NaN,0.652975
2,NaN,0.649659,NaN,0.652975
3,NaN,0.649659,NaN,0.652975
4,NaN,0.649659,NaN,0.652975


### Feature Scaling & Normalization

In [48]:
X_train.describe()

,danceability,energy,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,...,D,D#,E,F,F#,G,G#,A,A#,B
count,68400.000000,68400.000000,68400.000000,68400.000000,68400.000000,68400.000000,68400.000000,68400.000000,68400.000000,68400.000000,...,68400.000000,68400.000000,68400.000000,68400.000000,68400.000000,68400.000000,68400.000000,68400.000000,68400.00000,68400.000000
mean,0.567052,0.641239,-8.253307,0.639327,0.084536,0.315946,0.154460,0.214324,0.474524,122.028761,...,0.103947,0.031301,0.079459,0.081959,0.068582,0.115351,0.063070,0.100044,0.06617,0.081681
std,0.173204,0.251079,5.033128,0.480199,0.105831,0.332552,0.308156,0.191372,0.259062,29.964368,...,0.305194,0.174132,0.270456,0.274304,0.252744,0.319447,0.243091,0.300061,0.24858,0.273881
min,0.000000,0.000000,-49.531000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
25%,0.457000,0.473000,-10.008500,0.000000,0.035800,0.017200,0.000000,0.098100,0.261000,99.041500,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
50%,0.580000,0.684000,-7.001000,1.000000,0.048900,0.170000,0.000039,0.132000,0.464000,122.013000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
75%,0.695000,0.853000,-4.997000,1.000000,0.084300,0.599000,0.046200,0.275000,0.684000,140.051000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
max,0.983000,1.000000,4.532000,1.000000,0.963000,0.996000,1.000000,1.000000,0.995000,222.605000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000


In [49]:
# YOUR CODE HERE
X_train_features_norm = (X_train - X_train.mean())/X_train.std() 
X_test_features_norm = (Y_test - X_train.mean())/X_train.std()
#only train features shown since we're not supposed to look at test
X_train_features_norm.describe()

,danceability,energy,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,...,D,D#,E,F,F#,G,G#,A,A#,B
count,6.840000e+04,6.840000e+04,6.840000e+04,6.840000e+04,6.840000e+04,6.840000e+04,6.840000e+04,6.840000e+04,6.840000e+04,6.840000e+04,...,6.840000e+04,6.840000e+04,6.840000e+04,6.840000e+04,6.840000e+04,6.840000e+04,6.840000e+04,6.840000e+04,6.840000e+04,6.840000e+04
mean,2.929431e-17,-3.119532e-16,3.022923e-16,7.842979e-18,-1.006602e-16,-1.038805e-18,-1.869849e-18,-5.069369e-17,-3.064475e-17,2.043330e-16,...,-7.853367e-17,-2.295759e-17,-3.552714e-17,-5.204414e-17,-4.653847e-17,-8.310441e-19,-5.651100e-17,-3.037207e-17,-1.558208e-17,-1.952954e-17
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,...,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
min,-3.273905e+00,-2.553939e+00,-8.201201e+00,-1.331380e+00,-7.987817e-01,-9.500644e-01,-5.012398e-01,-1.119934e+00,-1.831698e+00,-4.072462e+00,...,-3.405939e-01,-1.797557e-01,-2.937968e-01,-2.987888e-01,-2.713496e-01,-3.610952e-01,-2.594510e-01,-3.334121e-01,-2.661902e-01,-2.982369e-01
25%,-6.353934e-01,-6.700669e-01,-3.487281e-01,-1.331380e+00,-4.605054e-01,-8.983432e-01,-5.012398e-01,-6.073187e-01,-8.242186e-01,-7.671532e-01,...,-3.405939e-01,-1.797557e-01,-2.937968e-01,-2.987888e-01,-2.713496e-01,-3.610952e-01,-2.594510e-01,-3.334121e-01,-2.661902e-01,-2.982369e-01
50%,7.475318e-02,1.703073e-01,2.488128e-01,7.510894e-01,-3.367228e-01,-4.388666e-01,-5.011142e-01,-4.301765e-01,-4.062375e-02,-5.259790e-04,...,-3.405939e-01,-1.797557e-01,-2.937968e-01,-2.987888e-01,-2.713496e-01,-3.610952e-01,-2.594510e-01,-3.334121e-01,-2.661902e-01,-2.982369e-01
75%,7.387114e-01,8.434033e-01,6.469748e-01,7.510894e-01,-2.226178e-03,8.511561e-01,-3.513159e-01,3.170606e-01,8.085923e-01,6.014557e-01,...,-3.405939e-01,-1.797557e-01,-2.937968e-01,-2.987888e-01,-2.713496e-01,-3.610952e-01,-2.594510e-01,-3.334121e-01,-2.661902e-01,-2.982369e-01
max,2.401494e+00,1.428877e+00,2.540231e+00,7.510894e-01,8.300660e+00,2.044953e+00,2.743865e+00,4.105500e+00,2.009075e+00,3.356528e+00,...,2.936005e+00,5.563024e+00,3.403664e+00,3.346797e+00,3.685229e+00,2.769312e+00,3.854236e+00,2.999247e+00,3.756658e+00,3.352990e+00


### Model 2

In [50]:
def build_model2(num_features, n_classes,learning_rate=0.01):
    """Build a TF logistic regression model using Keras.
    
    Args:
    learning_rate: The desired learning rate for SGD.
    
    Returns:
    model: A tf.keras model (graph).
    """
    # Random Seed + Clear Session
    tf.keras.backend.clear_session()
    np.random.seed(0)
    tf.random.set_seed(0)
    
    # Build a model using keras.Sequential.
    model = keras.Sequential(name = 'Genres')
    
    # Keras layers processing
    model.add(tf.keras.layers.InputLayer(input_shape=num_features))

    #adding extra layer
    model.add(keras.layers.Dense(
            units=256,
            activation = 'relu'))

    #add extra layer
    model.add(keras.layers.Dense(
            units=128,
            activation = 'relu'))
    
    # This layer constructs the linear set of parameters for each input feature
    # (as well as a bias), and applies a sigmoid to the result. The result is
    # binary logistic regression.
    model.add(keras.layers.Dense(
      units=n_classes,
      activation='softmax'))

    # Use the SGD optimizer as usual.
    optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate)
    
    # We specify the binary_crossentropy loss (equivalent to log loss).
    # Notice that we are including 'binary accuracy' as one of the metrics that we
    # ask Tensorflow to report when evaluating the model.
    model.compile(loss='sparse_categorical_crossentropy', 
                optimizer=optimizer, 
                metrics=['accuracy'])
    
    return model

In [51]:
model2 = build_model2(num_features = X_train.shape[1], n_classes = len(Y_train['music_category'].unique()))
model2.summary()

Model: "Genres"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 256)               5888      
                                                                 
 dense_1 (Dense)             (None, 128)               32896     
                                                                 
 dense_2 (Dense)             (None, 7)                 903       
                                                                 
Total params: 39687 (155.03 KB)
Trainable params: 39687 (155.03 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [52]:
# Fit the model.
history2 = model2.fit(
  x = X_train,   # training examples
  y = Y_train,   #labels
  epochs=10,             # number of passes through the training data
  batch_size=128,        # mini-batch size for SGD
  validation_split=0.1, # use a fraction of the examples for validation -- DO WE STILL NEED THIS IF VALIDATION SPLIT
  verbose=1             # display some progress output during training
  )

# Convert the return value into a DataFrame so we can see the train loss 
# and binary accuracy after every epoch.
history2 = pd.DataFrame(history2.history)
display(history)

Epoch 1/10
481/481 [==============================] - 1s 1ms/step - loss: 1.7202 - accuracy: 0.6449 - val_loss: 1.2715 - val_accuracy: 0.6530
Epoch 2/10
481/481 [==============================] - 1s 1ms/step - loss: 1.2382 - accuracy: 0.6497 - val_loss: 1.2096 - val_accuracy: 0.6530
Epoch 3/10
481/481 [==============================] - 1s 1ms/step - loss: 1.2181 - accuracy: 0.6497 - val_loss: 1.1998 - val_accuracy: 0.6530
Epoch 4/10
481/481 [==============================] - 1s 1ms/step - loss: 1.2137 - accuracy: 0.6497 - val_loss: 1.1966 - val_accuracy: 0.6530
Epoch 5/10
481/481 [==============================] - 1s 1ms/step - loss: 1.2123 - accuracy: 0.6497 - val_loss: 1.1955 - val_accuracy: 0.6530
Epoch 6/10
481/481 [==============================] - 1s 1ms/step - loss: 1.2118 - accuracy: 0.6497 - val_loss: 1.1950 - val_accuracy: 0.6530
Epoch 7/10
481/481 [==============================] - 1s 1ms/step - loss: 1.2116 - accuracy: 0.6497 - val_loss: 1.1948 - val_accuracy: 0.6530
Epoch 

,loss,accuracy,val_loss,val_accuracy
0,NaN,0.577047,NaN,0.652975
1,NaN,0.649659,NaN,0.652975
2,NaN,0.649659,NaN,0.652975
3,NaN,0.649659,NaN,0.652975
4,NaN,0.649659,NaN,0.652975


### Testing - Feature Selection

In [53]:
X_train.columns

Index(['danceability', 'energy', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'C',
       'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B'],
      dtype='object')

In [54]:
features = ['danceability','energy', 'loudness', 'valence', 'tempo']
X_train = X_train[features]

In [55]:
X_train.head()

,danceability,energy,loudness,valence,tempo
0,0.689,0.590,-9.272,0.872,93.914
1,0.584,0.626,-5.068,0.835,112.360
2,0.406,0.822,-4.172,0.534,127.688
3,0.728,0.514,-6.499,0.871,170.055
4,0.483,0.491,-5.389,0.400,111.956


In [56]:
model2 = build_model2(num_features = len(features), n_classes = len(Y_train['music_category'].unique()))
model2.summary()

Model: "Genres"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 256)               1536      
                                                                 
 dense_1 (Dense)             (None, 128)               32896     
                                                                 
 dense_2 (Dense)             (None, 7)                 903       
                                                                 
Total params: 35335 (138.03 KB)
Trainable params: 35335 (138.03 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [57]:
# Fit the model.
history2 = model2.fit(
  x = X_train,   # training examples
  y = Y_train,   #labels
  epochs=10,             # number of passes through the training data
  batch_size=128,        # mini-batch size for SGD
  validation_split=0.1, # use a fraction of the examples for validation -- DO WE STILL NEED THIS IF VALIDATION SPLIT
  verbose=1             # display some progress output during training
  )

# Convert the return value into a DataFrame so we can see the train loss 
# and binary accuracy after every epoch.
history2 = pd.DataFrame(history2.history)
display(history)

Epoch 1/10
481/481 [==============================] - 1s 1ms/step - loss: 1.7812 - accuracy: 0.6449 - val_loss: 1.2612 - val_accuracy: 0.6530
Epoch 2/10
481/481 [==============================] - 1s 1ms/step - loss: 1.2354 - accuracy: 0.6497 - val_loss: 1.2083 - val_accuracy: 0.6530
Epoch 3/10
481/481 [==============================] - 1s 1ms/step - loss: 1.2179 - accuracy: 0.6497 - val_loss: 1.1993 - val_accuracy: 0.6530
Epoch 4/10
481/481 [==============================] - 1s 1ms/step - loss: 1.2136 - accuracy: 0.6497 - val_loss: 1.1963 - val_accuracy: 0.6530
Epoch 5/10
481/481 [==============================] - 1s 1ms/step - loss: 1.2123 - accuracy: 0.6497 - val_loss: 1.1952 - val_accuracy: 0.6530
Epoch 6/10
481/481 [==============================] - 1s 1ms/step - loss: 1.2118 - accuracy: 0.6497 - val_loss: 1.1948 - val_accuracy: 0.6530
Epoch 7/10
481/481 [==============================] - 1s 1ms/step - loss: 1.2116 - accuracy: 0.6497 - val_loss: 1.1947 - val_accuracy: 0.6530
Epoch 

,loss,accuracy,val_loss,val_accuracy
0,NaN,0.577047,NaN,0.652975
1,NaN,0.649659,NaN,0.652975
2,NaN,0.649659,NaN,0.652975
3,NaN,0.649659,NaN,0.652975
4,NaN,0.649659,NaN,0.652975
